In [2]:
!nvidia-smi #1

Sat Feb 21 03:50:29 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   33C    P0             54W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
#2
!pip -q uninstall -y sentence-transformers fastai cudf-cu12 pylibcudf-cu12 rapids-dask-dependency rapids-logger || true

In [14]:
import wandb

try:
    wandb.finish()
except Exception as e:
    print("wandb.finish() skipped:", e)

In [4]:
#3
!pip -q install -U "transformers>=4.45,<5" "huggingface-hub>=0.23" "tokenizers>=0.20" \
  accelerate datasets peft bitsandbytes sentencepiece wandb rouge-score
!pip -q install -U deepspeed

In [5]:
#QCHECK
import torch, transformers, huggingface_hub, bitsandbytes
print("torch:", torch.__version__)
print("cuda:", torch.cuda.is_available(), torch.cuda.get_device_name(0))
print("transformers:", transformers.__version__)
print("hub:", huggingface_hub.__version__)
print("bnb:", bitsandbytes.__version__)

torch: 2.6.0+cu124
cuda: True NVIDIA A100-SXM4-80GB
transformers: 4.57.6
hub: 0.36.2
bnb: 0.49.2


In [5]:
#4
%cd /content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1/source_code
!mkdir -p finetuned_models
!ls -lah

/content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1/source_code
total 84K
drwxr-xr-x 3 root root 4.0K Feb 21 03:51 .
drwxr-xr-x 5 root root 4.0K Feb 21 03:47 ..
-rw-r--r-- 1 root root  783 Feb 21 03:47 config.json
drwxr-xr-x 2 root root 4.0K Feb 21 03:51 finetuned_models
-rw-r--r-- 1 root root  42K Feb 21 03:47 FinGPT-Assignment1.ipynb
-rw-r--r-- 1 root root 8.7K Feb 21 03:47 train_lora.py
-rw-r--r-- 1 root root  668 Feb 21 03:47 train.sh
-rw-r--r-- 1 root root 5.6K Feb 21 03:47 utils.py


In [6]:
#5
import os #refreshed keys

os.environ["HF_TOKEN"] = ""
os.environ["FINNHUB_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [7]:
#6
from huggingface_hub import login
login(token=os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [8]:
#7
!git clone https://github.com/AI4Finance-Foundation/STAT-GR5398-Spring-2026.git

Cloning into 'STAT-GR5398-Spring-2026'...
remote: Enumerating objects: 1100, done.
remote: Counting objects: 100% (797/797), done.
remote: Compressing objects: 100% (689/689), done.
remote: Total 1100 (delta 244), reused 436 (delta 102), pack-reused 303 (from 1)
Receiving objects: 100% (1100/1100), 80.54 MiB | 16.12 MiB/s, done.
Resolving deltas: 100% (307/307), done.


In [9]:
#8
!pip -q uninstall -y opencv-python-headless opencv-python

In [10]:
#9
%cd /content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1
!find . -maxdepth 4 -type d | egrep "dow|dji|fingpt|dataset|train|test" | head -n 120

/content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1


In [11]:
#10
!pip install --upgrade wandb

In [12]:
#11
import wandb

run = wandb.init(project="test-login-check")
print("Logged in as:", wandb.run.entity)
wandb.finish()

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Create a new API key at: https://wandb.ai/authorize?ref=models
wandb: Store your API key securely and do not share it.
wandb: Paste your API key and hit enter:

 ··········


wandb: ERROR Invalid API key: API key may only contain the letters A-Z, digits and underscores.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

...

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Create a new API key at: https://wandb.ai/authorize?ref=models
wandb: Store your API key securely and do not share it.
wandb: Paste your API key and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bx2233 (bx2233-columbia-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Logged in as: bx2233-columbia-university


In [24]:
#RE NEWRUN
import wandb
wandb.init(
    project="fingpt-forecaster",
    name="dow30_lora_v1_reconnect",
    reinit=True
)

In [1]:
import wandb
wandb.finish()

In [13]:
#QCHECK
import torch
print("torch:", torch.__version__)
print("cuda:", torch.cuda.is_available())

torch: 2.6.0+cu124
cuda: True


In [14]:
#12
import os, sys, importlib.util
print("cwd:", os.getcwd())
print("first sys.path:", sys.path[0])

spec = importlib.util.find_spec("torchvision")
print("torchvision spec:", spec)
print("torchvision origin:", spec.origin if spec else None)

cwd: /content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1
first sys.path: /content
torchvision spec: ModuleSpec(name='torchvision', loader=<_frozen_importlib_external.SourceFileLoader object at 0x78815c592510>, origin='/usr/local/lib/python3.11/dist-packages/torchvision/__init__.py', submodule_search_locations=['/usr/local/lib/python3.11/dist-packages/torchvision'])
torchvision origin: /usr/local/lib/python3.11/dist-packages/torchvision/__init__.py


In [15]:
#CHECK
!ls -lah | egrep -i "torchvision"
!find . -maxdepth 2 -iname "torchvision*" -print

In [16]:
#CHECK
import torch
print(torch.__version__)

2.6.0+cu124


In [17]:
#13
!pip -q uninstall -y torch torchvision torchaudio
!pip -q install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.5/768.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 137.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 135.7 MB/s eta 0:00:00


In [18]:
#14 RESTART SESSION BEFORE
import torch, torchvision
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
import torchvision.ops
print("torchvision ops OK")

torch: 2.6.0+cu124
torchvision: 0.21.0+cu124
torchvision ops OK


In [19]:
#15
%cd /content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1/source_code

!grep -n "prepare_model_for_int8_training" -n train_lora.py
!sed -i 's/prepare_model_for_int8_training/prepare_model_for_kbit_training/g' train_lora.py

!grep -n "prepare_model_for_" -n train_lora.py | head -n 20

/content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1/source_code
25:    prepare_model_for_int8_training,
174:    model = prepare_model_for_int8_training(model)
25:    prepare_model_for_kbit_training,
174:    model = prepare_model_for_kbit_training(model)


In [20]:
#CLEANUP
%cd /content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1/source_code

!sed -i 's/prepare_model_for_int8_training/prepare_model_for_kbit_training/g' train_lora.py

!grep -n "prepare_model_for_int8_training" train_lora.py || echo "✅ old name gone"
!grep -n "prepare_model_for_kbit_training" train_lora.py | head -n 20

/content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1/source_code
✅ old name gone
25:    prepare_model_for_kbit_training,
174:    model = prepare_model_for_kbit_training(model)


In [21]:
#16
!python train_lora.py --help | head -n 40

2026-02-21 03:54:16.364580: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-21 03:54:16.382424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771646056.404043    4568 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771646056.410426    4568 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-21 03:54:16.431768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [32]:
%%bash
apt-get update -qq
apt-get install -y -qq libopenmpi-dev openmpi-bin
pip -q install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.5 MB/s eta 0:00:00


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [33]:
!ls -lah /content
!ls -lah /content/STAT-GR5398-Spring-2026/FinGPT-Group/
!find /content -maxdepth 5 -name "train_lora.py"

total 28K
drwxr-xr-x 1 root root 4.0K Feb 21 00:16 .
drwxr-xr-x 1 root root 4.0K Feb 20 23:55 ..
drwxr-xr-x 1 root root 4.0K Jul 15  2025 .config
drwxr-xr-x 2 root root 4.0K Feb 21 00:09 finetuned_models
drwxr-xr-x 1 root root 4.0K Jul 15  2025 sample_data
drwxr-xr-x 6 root root 4.0K Feb 21 00:16 STAT-GR5398-Spring-2026
drwxr-xr-x 7 root root 4.0K Feb 21 00:17 wandb
total 16K
drwxr-xr-x 3 root root 4.0K Feb 21 00:16 .
drwxr-xr-x 6 root root 4.0K Feb 21 00:16 ..
drwxr-xr-x 5 root root 4.0K Feb 21 00:16 Assignment1
-rw-r--r-- 1 root root   25 Feb 21 00:16 README.md
/content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1/source_code/train_lora.py


In [34]:
%%bash
cd /content/STAT-GR5398-Spring-2026/FinGPT-Group/Assignment1/source_code
export DS_ACCELERATOR=cuda

python train_lora.py \
  --run_name dow30_lora_v1 \
  --base_model llama2 \
  --dataset fingpt-forecaster-dow30-20231231-20241231-1-4-06 \
  --max_length 4096 \
  --batch_size 1 \
  --gradient_accumulation_steps 16 \
  --learning_rate 5e-5 \
  --num_epochs 1 \
  --log_interval 10 \
  --warmup_ratio 0.03 \
  --scheduler constant \
  --evaluation_strategy steps \
  --eval_steps 50 \
  --ds_config config.json

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (

2026-02-21 00:22:34.466775: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-21 00:22:34.484789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771633354.506932    9322 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771633354.513340    9322 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-21 00:22:34.535145: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [35]:
!ls -lah

total 96K
drwxr-xr-x 6 root root 4.0K Feb 21 00:23 .
drwxr-xr-x 5 root root 4.0K Feb 21 00:16 ..
-rw-r--r-- 1 root root  783 Feb 21 00:16 config.json
drwxr-xr-x 4 root root 4.0K Feb 21 00:23 finetuned_models
-rw-r--r-- 1 root root  42K Feb 21 00:16 FinGPT-Assignment1.ipynb
drwxr-xr-x 4 root root 4.0K Feb 21 00:19 pretrained-models
drwxr-xr-x 2 root root 4.0K Feb 21 00:18 __pycache__
-rw-r--r-- 1 root root 8.7K Feb 21 00:18 train_lora.py
-rw-r--r-- 1 root root  668 Feb 21 00:16 train.sh
-rw-r--r-- 1 root root 5.6K Feb 21 00:16 utils.py
drwxr-xr-x 3 root root 4.0K Feb 21 00:23 wandb


In [36]:
!find . -maxdepth 3 -type d -name "checkpoint-*"
!find . -maxdepth 5 -type f -name "adapter_*" -o -name "*.safetensors" -o -name "*.bin" -o -name "trainer_state.json"

./finetuned_models/dow30_lora_v1_202602210023/checkpoint-50
./finetuned_models/dow30_lora_v1_202602210023/checkpoint-77
./finetuned_models/dow30_lora_v1_202602210023/adapter_config.json
./finetuned_models/dow30_lora_v1_202602210023/checkpoint-50/training_args.bin
./finetuned_models/dow30_lora_v1_202602210023/checkpoint-50/adapter_config.json
./finetuned_models/dow30_lora_v1_202602210023/checkpoint-50/trainer_state.json
./finetuned_models/dow30_lora_v1_202602210023/checkpoint-50/adapter_model.safetensors
./finetuned_models/dow30_lora_v1_202602210023/adapter_model.safetensors
./finetuned_models/dow30_lora_v1_202602210023/checkpoint-77/training_args.bin
./finetuned_models/dow30_lora_v1_202602210023/checkpoint-77/adapter_config.json
./finetuned_models/dow30_lora_v1_202602210023/checkpoint-77/trainer_state.json
./finetuned_models/dow30_lora_v1_202602210023/checkpoint-77/adapter_model.safetensors
./pretrained-models/models--deepseek-ai--DeepSeek-R1-Distill-Llama-8B/.no_exist/6a6f4aa4197940ad